# Belief Networks -  MNist Expert Generator

This document creates experts, CNN' which can categorize Mnist.
These experts can then be loaded into BN_datfile_generator.ipynb

### Loading the datasets

In [1]:
# Import Relevant Libraries
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models
import csv
from skimage.metrics import structural_similarity as ssim
from tensorflow.keras.models import load_model
from scipy.optimize import minimize


# Importing the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

/Users/christopherwaight/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


### Creating the experts

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import models, layers

# Random Seed
np.random.seed(12883)

# Create the model
model = models.Sequential([
    layers.Conv2D(2, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.BatchNormalization(),
    layers.Conv2D(2, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(2, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(20, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(20, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

model.summary()

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Training loop to save 10 different models
for epoch in range(1,11):  # From epoch 1 to 11 gives us 10
    print(f"Training epoch {epoch}")
    current_size = int(2048/(2**(epoch-1)))  # Halve the batch size every epoch
    model.fit(train_images, train_labels, epochs=1, batch_size=current_size, validation_split=0.25) # Train the model for 1 epoch
    model_path = f"mnist_expert{epoch}.keras"  # Path to save the model
    model.save(model_path)  # Save the model
    print(f"Model saved to {model_path}")

    # Evaluate the model on the test set
    test_loss, test_acc = model.evaluate(test_images, test_labels)
    print(f"Test accuracy after epoch {epoch}: {test_acc}")
